In [1]:
import time
from dataclasses import dataclass
import json
import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import  GemmaTokenizerFast, BitsAndBytesConfig, Gemma2ForCausalLM
from transformers.data.data_collator import pad_without_fast_tokenizer_warning

from transformers import  GemmaTokenizerFast, BitsAndBytesConfig, Gemma2ForCausalLM

/home/chenning/miniconda3/envs/llm_v1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = '../datas/train.csv'
test_path = '../datas/test.csv'
sample_path = '../datas/sample_submission.csv'
mapping_path = '../datas/misconception_mapping.csv'

m_map = pd.read_csv(mapping_path)
m_map.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


In [3]:
train_path = '../datas/data/train_data.json'
dev_path = '../datas/data/dev_data.json'
all_path = '../datas/data/all_data.json'

train_df = pd.read_json(train_path)
dev_df = pd.read_json(dev_path)
all_df = pd.read_json(all_path)
all_df = all_df[all_df.MisconceptionId != -1].reset_index(drop=True)
all_df['QuestionId'] = all_df['QuestionId'].astype(str)
all_df['QuestionId_Answer'] = all_df['QuestionId'] + "_" + all_df['InCorrectAnswer']


print(len(train_df), len(dev_df), len(all_df))
train_df.head()

3770 600 4370


,QuestionId_Answer,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName
0,0_D,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add..."
1,1_A,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...
2,1_B,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...
3,1_C,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...
4,2_A,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...


In [4]:
# 临时数据,no llm
import os
tmp_path = './tmp_data2'
os.makedirs(tmp_path, exist_ok=True)



In [6]:
## 构建召回数据
import copy 
llm_output = copy.deepcopy(all_df) 


tmp = []
for i in range(len(llm_output)):
    text = f"ConstructName:{llm_output.loc[i, 'ConstructName']}\nQuestionText:{llm_output.loc[i, 'QuestionText']}\nCorrectAnswerText:{llm_output.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{llm_output.loc[i, 'InCorrectAnswerText']}"
    
    tmp.append(text)
    
llm_output['text'] = tmp
print(llm_output['text'].values[0])
print(len(llm_output))
llm_output.to_parquet("./tmp_data1/recall_data_1.parquet", index=False)
llm_output.head()

ConstructName:Use the order of operations to carry out calculations involving powers
QuestionText:\[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
CorrectAnswerText:\( 3 \times(2+4)-5 \)
InCorrectAnswerText:Does not need brackets
4370


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,text
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,ConstructName:Use the order of operations to c...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,ConstructName:Simplify an algebraic fraction b...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,ConstructName:Simplify an algebraic fraction b...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,ConstructName:Simplify an algebraic fraction b...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,ConstructName:Calculate the range from a list ...


In [9]:
%%time
!/home/chenning/miniconda3/envs/llm_v2/bin/python run_similarity_search_nv.py\
    --gpu 3,4 \
    --input_file ./tmp_data1/recall_data_1.parquet \
    --output_file ./tmp_data1/recall_res_submission1.csv \
    --batch_size 5 \
    --recall_num 3000 \



/home/chenning/opensource_models/nvidia/NV-Embed-v2
Namespace(model_dir='/home/chenning/opensource_models/nvidia/NV-Embed-v2', lora_path='../recall_model_save/nv_round1_qlora_rerun/epoch_19_model/adapter.bin', input_file='./tmp_data1/recall_data_1.parquet', output_file='./tmp_data1/recall_res_submission1.csv', batch_size=5, gpu='3,4', max_length=512, seed=777, recall_num=3000, num_beam=1, spread_max_length=False, temp=1.03, use_lora=False)
  0%|                                                   | 0/874 [00:00<?, ?it/s]/home/chenning/.cache/huggingface/modules/transformers_modules/NV-Embed-v2/modeling_nvembed.py:350: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/home/chenning/miniconda3/envs/llm_v2/lib/python3.11/contextlib.py:105: FutureWarni

In [11]:
recall_res = pd.read_csv("./tmp_data1/recall_res_submission1.csv")[['QuestionId_Answer', 'MisconceptionId']]
print(recall_res.shape)
recall_res = recall_res.rename(columns={"MisconceptionId": "recallMisconceptionId"})
data_df2 = all_df.merge(recall_res, on='QuestionId_Answer', how='left')

data_df2.head()

(4370, 2)


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,recallMisconceptionId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,2488 2586 1345 2221 2518 871 1392 1999 1005 10...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,59 1593 891 1540 1755 1825 2398 29 1610 606 15...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,59 891 1593 1755 1540 1825 2398 29 1610 606 15...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,59 891 1593 1540 1825 1755 2398 29 1610 606 15...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,1073 1287 1677 2439 2471 1059 1349 397 1177 69...


In [13]:
save_path = './datas/no_llm_round1'
os.makedirs(save_path, exist_ok=True)


def save_json(json_path, json_df):
    df_str = json_df.to_json()
    print(df_str[0])
    with open(json_path, 'w') as f:
        f.write(df_str)
    print(f"save {json_path} finish")


save_json(os.path.join(save_path, "all_data.json"), data_df2)


dev_id = list(dev_df['QuestionId_Answer'].unique())

dev_df2 = data_df2[data_df2.QuestionId_Answer.isin(dev_id)].reset_index(drop=True)
train_df2 = data_df2[~data_df2.QuestionId_Answer.isin(dev_id)].reset_index(drop=True)
print(train_df2.shape, dev_df2.shape)
save_json(os.path.join(save_path, "dev_data.json"), dev_df2)

save_json(os.path.join(save_path, "train_data.json"), train_df2)



{
save ./datas/no_llm_round1/all_data.json finish
(3770, 14) (600, 14)
{
save ./datas/no_llm_round1/dev_data.json finish
{
save ./datas/no_llm_round1/train_data.json finish


In [14]:
dev_df2.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,recallMisconceptionId
0,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),A,\( \frac{62}{10} \),329,Does not understand a percentage is out of 100,6_A,847 122 318 1955 1786 1270 1914 2361 292 2368 ...
1,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),C,\( \frac{6}{2} \),847,Thinks they can convert a 2 digit percentage t...,6_C,847 122 318 1955 1270 292 1786 1973 2368 2361 ...
2,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),D,None of these,329,Does not understand a percentage is out of 100,6_D,847 318 122 1955 145 1786 804 2361 2368 292 32...
3,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),B,\( 0.432 \),2123,Divided by 100 rather than 10,7_B,1983 544 619 2133 2272 2200 2123 447 2002 964 ...
4,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),C,\( 33.2 \),2273,Subtracts instead of divides,7_C,1983 2272 2133 619 544 2200 447 2002 2123 964 ...


In [15]:
# 验证

import pandas as pd
pred_labels = list(dev_df2["recallMisconceptionId"].apply(lambda x: [int(y) for y in x.strip('[]').split(' ')]).values)
true_labels = list(dev_df2["MisconceptionId"].values)

def mean_average_precision(true_labels, predicted_labels_list, k=25):

    average_precisions = []
    tmp = []
    for true_label, predicted_labels in zip(true_labels, predicted_labels_list):
        if true_label in predicted_labels[:k]:
            idx = predicted_labels[:k].index(true_label)
            average_precisions.append(1/(idx + 1))
            tmp.append(1)
        else:
            average_precisions.append(0)
            tmp.append(0)
            
    print(sum(tmp) / len(tmp) if tmp else 0)
    return sum(average_precisions) / len(average_precisions) if average_precisions else 0
    
score = mean_average_precision(true_labels,pred_labels)
print(score)

0.7316666666666667
0.28230116543753947


# 构建训练召回数据集验证集不用构建


In [16]:
train_df = pd.read_json("./datas/no_llm_round1/train_data.json")
train_df.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,recallMisconceptionId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,2488 2586 1345 2221 2518 871 1392 1999 1005 10...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,59 1593 891 1540 1755 1825 2398 29 1610 606 15...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,59 891 1593 1755 1540 1825 2398 29 1610 606 15...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,59 891 1593 1540 1825 1755 2398 29 1610 606 15...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,1073 1287 1677 2439 2471 1059 1349 397 1177 69...


In [17]:
def get_neg(label, preds, k=5):
    preds_list = preds.split()
    label = int(label)
    
    samples = []
    for neg in preds_list:
        neg = int(neg)
        if neg == label:continue
        samples.append(mis_concepts[neg])
    return samples[0:k]
mis_concepts = m_map['MisconceptionName'].tolist()



In [18]:
total_examples = []
for i in range(len(train_df)):
    tmp = {}
    tmp['query'] = f"ConstructName:{train_df.loc[i, 'ConstructName']}\nQuestionText:{train_df.loc[i, 'QuestionText']}\nCorrectAnswerText:{train_df.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{train_df.loc[i, 'InCorrectAnswerText']}"
    tmp['pos'] = [train_df.loc[i, 'MisconceptionName']]
    tmp['neg'] = get_neg(int(train_df.loc[i, "MisconceptionId"]), train_df.loc[i, "recallMisconceptionId"], 50)
    total_examples.append(tmp)
s = set()
for k in total_examples:
    s |= set(k['neg'])

# assert len(s) == len(mis_concepts)

In [19]:
import json
filename = './datas/no_llm_round1/finetune_train.jsonl'
with open(filename, 'w', encoding='utf-8') as f:
    for entry in total_examples:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')  